<font color='darkorange'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, add your functions to the *apputil\.py* file and *app\.py* file as instructed. *These exercises use the same [Titanic dataset](https://www.kaggle.com/competitions/titanic/data) as the lab.*


## Exercise 1: Survival Patterns


For this exercise you will analyze survival patterns on the Titanic by looking at passenger class, sex, and age group. Name the function `survival_demographics()`.

1. Create a new column in the Titanic dataset that classifies passengers into age categories (i.e., a pandas `category` series). The categories should be:
    - Child (up to 12)
    - Teen (13–19)
    - Adult (20–59)
    - Senior (60+)  
  
	Hint: The `pd.cut()` function might come in handy here.

2. Group the passengers by class, sex, and age group.  

3. For each group, calculate:  
    - The total number of passengers, `n_passengers`
    - The number of survivors, `n_survivors`
    - The survival rate, `survival_rate`

4. Return a table that includes the results for *all* combinations of class, sex, and age group.  

5. Order the results so they are easy to interpret.  

6. Come up with a clear question that your results table makes you curious about (e.g., “Did women in first class have a higher survival rate than men in other classes?”). Write this question in your `app.py` file above the call to your visualization function, using `st.write("Your Question Here")`.
   
7. Create a Plotly visualization in a function named `visualize_demographic()` that directly addresses your question by returning a Plotly figure (e.g., `fig = px. ...`). You are free to choose the chart type that you think best communicates the findings. Be creative — try different approaches, compare them, and ensure that your chart clearly answers the question you posed.


In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

def survival_demographics():
    ages = [0, 12, 19, 59, float('inf')]
    labels = ['Child', 'Teen', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['Age'], bins=ages, labels=labels, right=True, include_lowest=True)
    df['age_group'] = df['age_group'].astype('category')

    age_grouped = df.groupby(['Pclass', 'Sex', 'age_group']).agg(
        n_passengers = ('PassengerId', 'count'),
        n_survivors = ('Survived', 'sum')
    )
    age_grouped['survival_rate'] = age_grouped['n_survivors'] / age_grouped['n_passengers']
    return age_grouped

survival_demographics()

def visualize_demographic():
    grouped_df = survival_demographics()
    print(grouped_df.columns)
    women_filtered = grouped_df[
        (grouped_df['Sex'] == 'female') & (grouped_df['age_group'].isin(['Teen','Adult']))
    ]
    #For answering this question, we will be using "women" as adult and teen females. Children will be any gender child. 
    women_fig = px.bar(women_filtered, 
                       x='age_group', 
                       y='Survived',
                       color= 'Pclass',
                       barmode= 'group'
    )

    women_fig.show()

visualize_demographic()


Index(['n_passengers', 'n_survivors', 'survival_rate'], dtype='object')


/tmp/ipykernel_1630/859893419.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipykernel_1630/859893419.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Exercise 2: Family Size and Wealth

Using the Titanic dataset, write a function named `family_groups()` to explore the relationship between family size, passenger class, and ticket fare.  

1. Create a new column in the Titanic dataset that represents the total family size for each passenger, `family_size`. Family size is defined as the number of siblings/spouses aboard plus the number of parents/children aboard, plus the passenger themselves.

2. Group the passengers by family size and passenger class. For each group, calculate:  
   - The total number of passengers, `n_passengers`
   - The average ticket fare, `avg_fare`
   - The minimum and maximum ticket fares (to capture variation in wealth), `min_fare` and `max_fare`

3. Return a table with these results, sorted so that the values are clear and easy to interpret (for example, by class and then family size).

4. Write a function called `last_names()` that extracts the last name of each passenger from the `Name` column, and returns the count for each last name (i.e., a pandas series with last name as index, and count as value). Does this result agree with that of the data table above? Share your findings in your app using `st.write`.

5. Just like you did in Exercise 1, come up with a clear question that your results makes you curious about. Write this question in your app.py file above the call to your visualization function. Then, create a Plotly visualization in a function named `visualize_families()` that directly addresses your question. As in Exercise 1 you are free to choose the chart type that you think best communicates the findings.

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

def family_groups():
    df['family_size'] = df['Parch'] + df['SibSp'] + 1
    family_grouped = df.groupby(['Pclass', 'family_size']).agg(
        n_passengers = ('PassengerId', 'count'),
        avg_fare = ('Fare', 'mean'),
        min_fare = ('Fare', 'min'),
        max_fare = ('Fare', 'max')
    )
    return family_grouped

print(family_groups())

def last_names():
    last_names = df['Name'].str.split(',', expand=True)[0].str.strip()
    return last_names.value_counts()

print(last_names())

                    n_passengers    avg_fare  min_fare  max_fare
Pclass family_size                                              
1      1                     109   63.672514    0.0000  512.3292
       2                      70   91.848039   29.7000  512.3292
       3                      24   95.681075   26.2833  211.5000
       4                       7  133.521429  120.0000  151.5500
       5                       2  262.375000  262.3750  262.3750
       6                       4  263.000000  263.0000  263.0000
2      1                     104   14.066106    0.0000   73.5000
       2                      34   24.682962   11.5000   33.0000
       3                      31   31.693819   13.0000   73.5000
       4                      13   36.575969   11.5000   65.0000
       5                       1   23.000000   23.0000   23.0000
       6                       1   18.750000   18.7500   18.7500
3      1                     324    9.272052    0.0000   56.4958
       2                 

## Bonus Question

Add a new column, `older_passenger`, to the Titanic dataset that indicates whether each passenger’s age is above the median age for *their* passenger class. So, suppose row $x$ is in passenger class 2. Then, a value of `True` at row $x$ would indicate that passenger older than 50% of class 2 passengers, and `False` would indicate that they younger.

- You should use pandas functions to accomplish this.
- The new column should contain Boolean values (True if the age is above the median, False if less than or equal to).
- Return the updated table in the function `determine_age_division()`

Once you’ve created this column, consider how this age division relates to your analysis above. Try to visualize this analysis in Plotly using the function name `visualize_age_division()`.